In [322]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.rcParams['figure.figsize'] = (20.0, 10.0)

## DATA COLLECTION

In [323]:
data = pd.read_csv('data/Iris.csv')
data.drop('Id', axis=1, inplace=True)
data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [324]:
print(data['Species'].unique())

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [325]:
header = data.columns
header

Index(['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [326]:
from sklearn.model_selection import train_test_split

In [327]:
data = data.to_numpy()
data, test_data = train_test_split(data, test_size=0.3)

In [328]:
def class_counts(data):
    """Counts the number of each type of example in a dataset."""
    counts = {}
    for row in data:
#         print(row)
        label = row[-1]
#         print(label)
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [329]:
class_counts(data)

{'Iris-setosa': 32, 'Iris-versicolor': 32, 'Iris-virginica': 41}

In [330]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [331]:
x, y, z = 12, 614.26, 'asfan'
print(is_numeric(x), is_numeric(y), is_numeric(z))

True True False


# Decision Tree

In [332]:
class Question:
    """A Questiong is used to partition a dataset.
    
    This class just records a 'column number' (e.g. 0 for SepalLengthCm) and a
    'column value' (e.g. 5.1). The 'match' method is used to compare the feature
    value in an example to the feature value stored in the question.
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
        
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return (f"Is {header[self.column]} {condition} {str(self.value)}?")

In [333]:
Question(1, 3)

Is SepalWidthCm >= 3?

In [334]:
q = Question(4, 'Iris-setosa')
q.match(data[0])

True

In [335]:
def partition(data, question):
    """Partitions a dataset.
    
    For each row in the dataset, check if it matches the question. If
    so, add it to the 'true rows', otherwise, add it to 'false rows'.
    """
    
    true_rows, false_rows = [], []
    for row in data:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [336]:
true_rows, false_rows = partition(data, Question(1, 3))

In [337]:
def gini(data):
    """Calculate the Gini Impurity for a list of rows."""
    
    counts = class_counts(data)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(data))
        impurity -= prob_of_lbl**2
    return impurity

In [338]:
gini(data)

0.6617687074829932

In [339]:
def info_gain(left, right, current_uncertainity):
    """Information Gain.
    
    The Uncertainity of the starting node, minus the weighted impurity of
    two child nodes.
    """
    
    p = float(len(left)) / (len(left) + len(right))
    return (current_uncertainity - p * gini(left) - (1 - p) * gini(right))

In [340]:
current_uncertainity = gini(data)
current_uncertainity

0.6617687074829932

In [341]:
from pprint import pprint as pp
true_rows, false_rows = partition(data, Question(2, 1))
# print(true_rows)
info_gain(true_rows, false_rows, current_uncertainity)
# true_rows = pd.DataFrame(true_rows)
# true_rows
# true_rows.value_counts()

0.0

In [342]:
def find_best_split(data):
    """Find the best question to ask by iterating over every feature/value
    and calculating the information gain."""
    best_gain = 0
    best_question = None
    n_features = len(data[0]) - 1
    current_uncertainity = gini(data)
    
    for col in range(n_features):
        
        values = set([row[col] for row in data])
        
        for val in values:
            question = Question(col, val)
            
            true_rows, false_rows = partition(data, question)
            
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            
            if gain == 0:
                continue
            
            if gain >= best_gain:
                best_gain, best_question = gain, question
                
    return best_gain, best_question

In [343]:
best_gain, best_question = find_best_split(data)
best_question, best_gain

(Is PetalWidthCm >= 1.0?, 0.31943341720249746)

In [344]:
class Leaf:
    """A Leaf node classifies data.
    
    This holds a dictionary of class (e.g. 'Iris-setosa') -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [345]:
class Decision_Node:
    """A Decision Node asks a question.
    
    This holds a refrence to the question, and to the two child nodes.
    """
    
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [346]:
def build_tree(data):
    """Builds the tree.
    
    Rules of recursion: 1) Believe that it works. 2) Start by checking for
    the base case (no further information gain). 3) Prepare for gaint stack traces.
    """
    
    gain, question = find_best_split(data)
    
    if gain == 0:
        return Leaf(data)
    
    true_rows, false_rows = partition(data, question)
    
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    
    return Decision_Node(question, true_branch, false_branch)

In [347]:
def print_tree(node, spacing=""):

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [348]:
my_tree = build_tree(data)

In [349]:
print_tree(my_tree)

Is PetalWidthCm >= 1.0?
--> True:
  Is PetalLengthCm >= 4.8?
  --> True:
    Is PetalWidthCm >= 1.8?
    --> True:
      Is PetalLengthCm >= 4.9?
      --> True:
        Predict {'Iris-virginica': 35}
      --> False:
        Is SepalWidthCm >= 3.2?
        --> True:
          Predict {'Iris-versicolor': 1}
        --> False:
          Predict {'Iris-virginica': 1}
    --> False:
      Is PetalLengthCm >= 5.6?
      --> True:
        Predict {'Iris-virginica': 2}
      --> False:
        Is PetalWidthCm >= 1.6?
        --> True:
          Predict {'Iris-versicolor': 2}
        --> False:
          Is PetalLengthCm >= 5.0?
          --> True:
            Predict {'Iris-virginica': 2}
          --> False:
            Predict {'Iris-versicolor': 1}
  --> False:
    Is PetalWidthCm >= 1.7?
    --> True:
      Predict {'Iris-virginica': 1}
    --> False:
      Predict {'Iris-versicolor': 28}
--> False:
  Predict {'Iris-setosa': 32}


In [350]:
def classify(row, node):
    
    if isinstance(node, Leaf):
        return node.predictions
    
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [351]:
classify(data[0], my_tree)

{'Iris-setosa': 32}

In [352]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] /total * 100)) + "%"
    return probs

In [353]:
print_leaf(classify(data[65], my_tree))

{'Iris-virginica': '100%'}

In [354]:
import operator
def predict(data, node):
    classes = classify(data, node)
    return max(classes.items(), key=operator.itemgetter(1))[0]

## Evaluation

In [355]:
from sklearn.metrics import accuracy_score

In [356]:
y_pred = []
for row in test_data:
    y_pred.append(predict(row, my_tree))

In [357]:
y_true = [row[-1] for row in test_data]

In [358]:
accuracy_score(y_true, y_pred)

1.0